In [1]:
# Taken from https://www.kaggle.com/code/suvroo/complete-nlp-pipeline#RoPE-(Robust-Positional-Embeddings)

from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score,confusion_matrix
import re
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# research-methods/imdb/IMDB_dataset.csv
df = pd.read_csv('imdb/IMDB_dataset.csv')
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [2]:
df.duplicated().sum()

418

In [3]:
df.drop_duplicates(inplace=True)

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/saarbu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text
df['review'] = df['review'].apply(remove_tags)
df_post_tags = df.copy()

sw_list = stopwords.words('english')
df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

df['review'] = df['review'].apply(lambda x:x.lower())

In [6]:
df.to_csv("imdb/pre_proccessed_imdb.csv", index=False)